# T2.1 - Solution
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [17]:
# Pacotes usados
import pandas as pd
from github import Github
import time


In [35]:
git = Github("leokaplan",,per_page=100)
print(git.get_rate_limit())
print(time.localtime(git.rate_limiting_resettime))
print(git.per_page)
commits_pag = git.get_user('torvalds').get_repo('linux').get_commits()

RateLimit(rate=Rate(remaining=4703, limit=5000))
time.struct_time(tm_year=2018, tm_mon=6, tm_mday=18, tm_hour=21, tm_min=0, tm_sec=31, tm_wday=0, tm_yday=169, tm_isdst=0)
100


In [29]:
print(commits_pag.totalCount)

None


In [33]:
commits = []
for c in commits_pag:
    commits.insert(len(commits),c)
    if(len(commits)%100==0):
        print(len(commits))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000


KeyboardInterrupt: 

In [41]:
for commit in commits[0:3000]:
    loc = commit.commit.message.split(' ')[0]
    typ = commit.commit.message.split(' ')[1]
    auth = commit.author
    committer = commit.committer
    files = commit.files

NamedUser(login="torvalds")
MAINTAINERS:
Add
NamedUser(login="torvalds")
Fix
Documentation
NamedUser(login="davem330")
net:
stmmac:
NamedUser(login="davem330")
l2tp:
filter
NamedUser(login="davem330")
l2tp:
reject
NamedUser(login="davem330")
mlxsw:
spectrum_switchdev:
NamedUser(login="davem330")
stmmac:
added
NamedUser(login="axboe")
block:
remov
NamedUser(login="jmberg")
cfg80211:
fix
NamedUser(login="jmberg")
nl80211:
fix
NamedUser(login="davem330")
l2tp:
clean
NamedUser(login="davem330")
l2tp:
prevent
NamedUser(login="davem330")
l2tp:
only
NamedUser(login="davem330")
l2tp:
fix
NamedUser(login="davem330")
net:
emaclite:
NamedUser(login="davem330")
net:
emaclite:
NamedUser(login="davem330")
net:
emaclite:
NamedUser(login="davem330")
net:
emaclite:
NamedUser(login="davem330")
net:
qcom/emac:
NamedUser(login="torvalds")
lib/test_printf.c:
call
NamedUser(login="torvalds")
hexagon:
drop
NamedUser(login="torvalds")
hexagon:
fix
NamedUser(login="torvalds")
mm:
fix
NamedUser(login="torvalds"

In [3]:
# Carregando dados do data set
commits = pd.read_json('data.json')
commits

,author,comments_url,commit,committer,html_url,node_id,parents,sha,url
0,"{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://api.github.com/repos/torvalds/linux/co...,"{'author': {'name': 'Linus Torvalds', 'email':...","{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://github.com/torvalds/linux/commit/28374...,MDY6Q29tbWl0MjMyNTI5ODoyODM3NDYxZGJlNmY0YTlhY2...,[{'sha': 'f3b5020e168a383496e3096b754a1ad17f09...,2837461dbe6f4a9acc0d86f88825888109211c99,https://api.github.com/repos/torvalds/linux/co...
1,"{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://api.github.com/repos/torvalds/linux/co...,"{'author': {'name': 'Linus Torvalds', 'email':...","{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://github.com/torvalds/linux/commit/f3b50...,MDY6Q29tbWl0MjMyNTI5ODpmM2I1MDIwZTE2OGEzODM0OT...,[{'sha': '4b4bb99b6246007eccbc0731eae0b6742c24...,f3b5020e168a383496e3096b754a1ad17f09df36,https://api.github.com/repos/torvalds/linux/co...
2,"{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://api.github.com/repos/torvalds/linux/co...,"{'author': {'name': 'Linus Torvalds', 'email':...","{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://github.com/torvalds/linux/commit/4b4bb...,MDY6Q29tbWl0MjMyNTI5ODo0YjRiYjk5YjYyNDYwMDdlY2...,[{'sha': '9bca19a01d50143b736f0f59eb3ccc05b110...,4b4bb99b6246007eccbc0731eae0b6742c24efa5,https://api.github.com/repos/torvalds/linux/co...
3,"{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://api.github.com/repos/torvalds/linux/co...,"{'author': {'name': 'Linus Torvalds', 'email':...","{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://github.com/torvalds/linux/commit/9bca1...,MDY6Q29tbWl0MjMyNTI5ODo5YmNhMTlhMDFkNTAxNDNiNz...,[{'sha': '463f202172c31b9c36278001cabfbad4e12d...,9bca19a01d50143b736f0f59eb3ccc05b1106172,https://api.github.com/repos/torvalds/linux/co...
4,"{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://api.github.com/repos/torvalds/linux/co...,"{'author': {'name': 'Linus Torvalds', 'email':...","{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://github.com/torvalds/linux/commit/463f2...,MDY6Q29tbWl0MjMyNTI5ODo0NjNmMjAyMTcyYzMxYjljMz...,[{'sha': '050e9baa9dc9fbd9ce2b27f0056990fc9e0a...,463f202172c31b9c36278001cabfbad4e12da42e,https://api.github.com/repos/torvalds/linux/co...
5,"{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://api.github.com/repos/torvalds/linux/co...,"{'author': {'name': 'Linus Torvalds', 'email':...","{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://github.com/torvalds/linux/commit/050e9...,MDY6Q29tbWl0MjMyNTI5ODowNTBlOWJhYTlkYzlmYmQ5Y2...,[{'sha': 'be779f03d563981c65cc7417cc5e0dbbc5b8...,050e9baa9dc9fbd9ce2b27f0056990fc9e0a08a0,https://api.github.com/repos/torvalds/linux/co...
6,"{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://api.github.com/repos/torvalds/linux/co...,"{'author': {'name': 'Linus Torvalds', 'email':...","{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://github.com/torvalds/linux/commit/be779...,MDY6Q29tbWl0MjMyNTI5ODpiZTc3OWYwM2Q1NjM5ODFjNj...,[{'sha': 'd290ef9305ebaaac884ae0350bfc243dd01d...,be779f03d563981c65cc7417cc5e0dbbc5b89d30,https://api.github.com/repos/torvalds/linux/co...
7,"{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://api.github.com/repos/torvalds/linux/co...,"{'author': {'name': 'Linus Torvalds', 'email':...","{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://github.com/torvalds/linux/commit/d290e...,MDY6Q29tbWl0MjMyNTI5ODpkMjkwZWY5MzA1ZWJhYWFjOD...,[{'sha': 'd09fcecb0c797b884ce65daa37c121a2786b...,d290ef9305ebaaac884ae0350bfc243dd01d354d,https://api.github.com/repos/torvalds/linux/co...
8,"{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://api.github.com/repos/torvalds/linux/co...,"{'author': {'name': 'Linus Torvalds', 'email':...","{'login': 'torvalds', 'id': 1024025, 'node_id'...",https://github.com/torvalds/linux/commit/d09fc...,MDY6Q29tbWl0MjMyNTI5ODpkMDlmY2VjYjBjNzk3Yjg4NG...,[{'sha': 'f5b7769eb0400ec5217

## Análise prévia dos dados:

# Perguntas:
## 1) ???